# **AutoGen Core**

* Something a little different.  
* This is agnostic to the underlying Agent framework.  
* You can use Autogen Agentchat, or you can use something else; It's an  Agent Interaction framework.  
+ From that point of view, it's positioned similarly to LangGraph.

#### **The Fundamental Principle**

+ Autogenc Core decouples an agent's logic from how messages are delivered.  
+ The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.  
+ The communication infrastructure is called a **Runtime**.  
+ There are two types: **Standalone** and **Distributed**.  
+ Today, we'll use a standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.

In [81]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)

True

#### **Step 1**: First we define our Message Object  
Whatever structure we want for our messages in our Agent framework  

* a `Message` data class, which defines the messages that are passed between the agents.

In [5]:
# Let's have a simple one

@dataclass
class Message:
    content: str

#### **Step 2**: Now we define our Agent  
A subclass of RoutedAgent.  
* Every Agent has an Agent ID which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` goves it its unique identifier  

* Any method with the `@message_handler` decorated will have the opportunity to receive messages.

In [7]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__(("Simple"))

    
    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")

---

> You might have already noticed, the agents’ logic, whether it is using model or code executor, is completely decoupled from how messages are delivered. This is the core idea: the framework provides a communication infrastructure, and the agents are responsible for their own logic. We call the communication infrastructure an **Agent Runtim**e.

> Agent runtime is a key concept of this framework. Besides delivering messages, it also manages agents’ lifecycle. So the creation of agents are handled by the runtime.

> The following code shows how to register and run the agents using SingleThreadedAgentRuntime, a local embedded agent runtime implementation.
---

#### **Step 3**: Let's create a Standalone runtime and register our agent type:

In [8]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

#### **Step 4:** Alright, let's start a runtime and send a message

In [9]:
runtime.start()

In [10]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


In [11]:
await runtime.stop()
await runtime.close()

#### Now, something more interesting!

We'll use an AgentChat Assistant.

In [24]:
class MyLLMAgent(RoutedAgent):
    
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model='gpt-4.1-nano')
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source='user')
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [25]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "MyLLMAgent", lambda: MyLLMAgent())

AgentType(type='MyLLMAgent')

In [28]:
runtime.start()

response = await runtime.send_message(Message("Hi there!"), AgentId("MyLLMAgent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
reponse = await runtime.send_message(Message(response.content), AgentId("MyLLMAgent", "default"))




MyLLMAgent received message: Hi there!
MyLLMAgent responded: Hello again! How can I help you today?
>>> Hello again! How can I help you today?
>>> This is simple_agent-default. You said 'Hello again! How can I help you today?' and I disagree.
MyLLMAgent received message: This is simple_agent-default. You said 'Hello again! How can I help you today?' and I disagree.
MyLLMAgent responded: Thank you for the feedback. How would you like me to respond?


In [ ]:
await runtime.stop()
await runtime.close()

#### Let's get 3 Agents busy this time!

In [90]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.5)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [91]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)

In [92]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [93]:

agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: scissors
Player 2: rock
Player 2 wins because rock beats scissors. TERMINATE


In [94]:

await runtime.stop()
await runtime.close()